In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/u

In [3]:
# Start a new session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Import your data from S3
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [5]:
# Recreate the vine_df dataframe
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R33UPQQUZQEM8|          5|            3|          3|   N|                Y|
|R3IKTNQQPD9662|          5|            0|          0|   N|                Y|
|R3U27V5QMCP27T|          5|            1|          1|   N|                Y|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|                Y|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [6]:
# Filter out any rows that had less than 20 total votes
vine_filter = vine_df.filter(vine_df.total_votes >= '20')
vine_filter.sort(['total_votes']).show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R25L560LHPHE2F|          1|           15|         20|   N|                N|
| RTIJFEYV3NIAT|          5|           15|         20|   N|                Y|
|R1WCLX1DYWRF6I|          1|           19|         20|   N|                Y|
|R2QQJIJZIUSQBH|          1|           16|         20|   N|                N|
|R18ZPPQ2CZKG41|          5|           16|         20|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
# Filter out any rows that had less thatn 50% of their total votes identified as helpful
vine_filter_2 = vine_filter.withColumn('helpful_rate', vine_filter['helpful_votes']/vine_filter['total_votes'])
vine_filter_help = vine_filter_2.filter(vine_filter_2.helpful_rate >= '.50').sort(['helpful_rate'], ascending=True)
vine_filter_help.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_rate|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
|R1L3VZLFY0Y0G5|          1|           11|         22|   N|                N|         0.5|
| R1ELR2Y33KD34|          3|           10|         20|   N|                N|         0.5|
|R241G74TIONA2K|          3|           17|         34|   N|                N|         0.5|
|R1WMQ595AF6N7M|          2|           10|         20|   N|                Y|         0.5|
|R1JOOLO8GN74AK|          1|           17|         34|   N|                Y|         0.5|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
only showing top 5 rows



In [8]:
# Create a dataframe that filters out the rows that were not paid vine subscribers
vine_filter_paid = vine_filter_help.filter(vine_filter_help.vine == 'Y')
vine_filter_paid.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      helpful_rate|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2Y8EYZCTO7W1X|          3|           11|         22|   Y|                N|               0.5|
|R1F8T2Z7U3N6HL|          3|           11|         20|   Y|                N|              0.55|
|R1QFLL874C1LYW|          1|           15|         27|   Y|                N|0.5555555555555556|
| R9PY1FWPZ7CFJ|          2|           13|         22|   Y|                N|0.5909090909090909|
|R160Z7M8HOM75I|          2|           13|         20|   Y|                N|              0.65|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
only showing top 5 rows



In [9]:
# Create a complimentary dataframe that filters out, instead, all of the rows that were part of the paid subscription
vine_filter_unpaid = vine_filter_help.filter(vine_filter_help.vine == 'N')
vine_filter_unpaid.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_rate|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
|R1L3VZLFY0Y0G5|          1|           11|         22|   N|                N|         0.5|
| R1ELR2Y33KD34|          3|           10|         20|   N|                N|         0.5|
|R241G74TIONA2K|          3|           17|         34|   N|                N|         0.5|
|R1WMQ595AF6N7M|          2|           10|         20|   N|                Y|         0.5|
|R1JOOLO8GN74AK|          1|           17|         34|   N|                Y|         0.5|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
only showing top 5 rows



In [27]:
# Import sql function count
from pyspark.sql.functions import count

# Identify the number of star ratings for each of the five star types for both the paid and unpaid filtered dataframes 
total_star_paid = vine_filter_paid.groupby("star_rating").agg(count("star_rating").alias("count(star_rating)")).withColumnRenamed("count(star_rating)", "total_star_reviews_paid").sort(['star_rating'], ascending=True)
total_star_paid.show()

total_star_unpaid = vine_filter_unpaid.groupby("star_rating").agg(count("star_rating").alias("count(star_rating)")).withColumnRenamed("count(star_rating)", "total_star_reviews_unpaid").sort(['star_rating'], ascending=True)
total_star_unpaid.show()

+-----------+-----------------------+
|star_rating|total_star_reviews_paid|
+-----------+-----------------------+
|          1|                      3|
|          2|                      6|
|          3|                     17|
|          4|                     14|
|          5|                      9|
+-----------+-----------------------+

+-----------+-------------------------+
|star_rating|total_star_reviews_unpaid|
+-----------+-------------------------+
|          1|                    24624|
|          2|                     9489|
|          3|                    13862|
|          4|                    25364|
|          5|                    78061|
+-----------+-------------------------+



In [28]:
# Identify the total number of paid and unpaid responses for the filtered dataframes
total_paid = vine_filter_paid.count()
total_unpaid = vine_filter_unpaid.count()

In [29]:
# Create a new column that shows what percentage of the total responses for that filtered dataframes each of the five star ratings received
percent_paid = total_star_paid.withColumn('percent_of_total_paid',(total_star_paid['total_star_reviews_paid'] / total_paid) * 100)
percent_paid.show()

percent_unpaid = total_star_unpaid.withColumn('percent_of_total_unpaid',(total_star_unpaid['total_star_reviews_unpaid'] / total_unpaid) * 100)
percent_unpaid.show()

+-----------+-----------------------+---------------------+
|star_rating|total_star_reviews_paid|percent_of_total_paid|
+-----------+-----------------------+---------------------+
|          1|                      3|    6.122448979591836|
|          2|                      6|   12.244897959183673|
|          3|                     17|    34.69387755102041|
|          4|                     14|    28.57142857142857|
|          5|                      9|   18.367346938775512|
+-----------+-----------------------+---------------------+

+-----------+-------------------------+-----------------------+
|star_rating|total_star_reviews_unpaid|percent_of_total_unpaid|
+-----------+-------------------------+-----------------------+
|          1|                    24624|      16.26420079260238|
|          2|                     9489|      6.267503302509908|
|          3|                    13862|      9.155878467635404|
|          4|                    25364|     16.752972258916778|
|          

In [30]:
# Filter our all of the other percentages to show specifically how many responses and what percentage the 
# five star ratings received for each of the two filtered dataframes
percent_five_paid = percent_paid.filter(percent_paid.star_rating == '5')
percent_five_paid.show()

percent_five_unpaid = percent_unpaid.filter(percent_unpaid.star_rating == '5')
percent_five_unpaid.show()


+-----------+-----------------------+---------------------+
|star_rating|total_star_reviews_paid|percent_of_total_paid|
+-----------+-----------------------+---------------------+
|          5|                      9|   18.367346938775512|
+-----------+-----------------------+---------------------+

+-----------+-------------------------+-----------------------+
|star_rating|total_star_reviews_unpaid|percent_of_total_unpaid|
+-----------+-------------------------+-----------------------+
|          5|                    78061|      51.55944517833554|
+-----------+-------------------------+-----------------------+

